In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Raw_Data/google-global-user-data-requests.csv")
df = df[df['Country/Region'] == "United States"]
df.tail(8)

,Period Ending,Country/Region,CLDR Territory Code,Legal Process,User Data Requests,Percentage disclosed,Accounts
2249,2020-06-30,United States,US,Wiretap Orders,2,50.0,4.0
2421,2020-12-31,United States,US,Emergency Disclosure Requests,1399,75.0,2197.0
2422,2020-12-31,United States,US,Other Court Orders,2297,84.0,9258.0
2423,2020-12-31,United States,US,Pen Register Orders,421,84.0,1574.0
2424,2020-12-31,United States,US,Preservation Requests,15502,NaN,42699.0
2425,2020-12-31,United States,US,Search Warrants,18349,81.0,28612.0
2426,2020-12-31,United States,US,Subpoenas,16587,82.0,42358.0
2427,2020-12-31,United States,US,Wiretap Orders,2,100.0,2.0


In [6]:
df.rename(columns = {
    "Period Ending": "period_ending",
    "User Data Requests": "total_requests",
    "Accounts": "total_requests_accounts",
    "Legal Process": "legal_process"
}, inplace=True)
df['half'] = [1 if "-06-" in period else 2 for period in df.period_ending]
df['year'] = [int(y[0:4]) for y in df.period_ending]

df = df[df.year > 2012]

print(len(df[
        np.array(df.legal_process=="Pen Register Orders") |
        np.array(df.legal_process=="Other Court Orders")
    ].groupby(["year", "half"]).sum()["total_requests_accounts"].values.tolist()))

print(len([r for r in df[df.legal_process=="Search Warrants"].half]))

16
16


In [9]:
reorg = pd.DataFrame.from_dict({
    "half": [r for r in df[df.legal_process=="Search Warrants"].half],
    "year": [r for r in df[df.legal_process=="Search Warrants"].year],
    "warrants": df[df.legal_process=="Search Warrants"]["total_requests"].values.tolist(),
    "warrant_accounts": df[df.legal_process=="Search Warrants"]["total_requests_accounts"].values.tolist(),
    "subpoenas": df[df.legal_process=="Subpoenas"]["total_requests"].values.tolist(),
    "subpoena_accounts": df[df.legal_process=="Subpoenas"]["total_requests_accounts"].values.tolist(),
    "emergency_requests": df[df.legal_process=="Emergency Disclosure Requests"]["total_requests"].values.tolist(),
    "emergency_request_accounts": df[
        df.legal_process=="Emergency Disclosure Requests"
    ][
        "total_requests_accounts"
    ].values.tolist(),
    "court_orders": df[
        np.array(df.legal_process=="Pen Register Orders") |
        np.array(df.legal_process=="Other Court Orders")
    ].groupby(["year", "half"]).sum()["total_requests"].values.tolist(),
    "court_order_accounts": df[
        np.array(df.legal_process=="Pen Register Orders") |
        np.array(df.legal_process=="Other Court Orders")
    ].groupby(["year", "half"]).sum()["total_requests_accounts"].values.tolist()
})

reorg['total_requests'] = reorg.apply(lambda r: r.subpoenas + r.warrants, axis=1)
reorg['total_requests_accounts'] = reorg.apply(lambda r: r.subpoena_accounts + r.warrant_accounts, axis=1)

reorg["source"] = "https://transparencyreport.google.com/?hl=en"
reorg.tail()

,half,year,warrants,warrant_accounts,subpoenas,subpoena_accounts,emergency_requests,emergency_request_accounts,court_orders,court_order_accounts,total_requests,total_requests_accounts,source
11,2,2018,8687,24994.0,11099,26669.0,994,1749.0,1963,9432.0,19786.0,51663.0,https://transparencyreport.google.com/?hl=en
12,1,2019,14664,23831.0,12946,35167.0,1184,2100.0,2439,13521.0,27610.0,58998.0,https://transparencyreport.google.com/?hl=en
13,2,2019,14652,21854.0,12110,28614.0,1246,2150.0,2367,13308.0,26762.0,50468.0,https://transparencyreport.google.com/?hl=en
14,1,2020,19783,28865.0,15537,38983.0,1324,2137.0,2889,14670.0,35320.0,67848.0,https://transparencyreport.google.com/?hl=en
15,2,2020,18349,28612.0,16587,42358.0,1399,2197.0,2718,10832.0,34936.0,70970.0,https://transparencyreport.google.com/?hl=en


In [10]:
reorg.to_csv("../Clean_Data/Google.csv", index=False)